# Pre Processing Step
We want to process the data for LDA like:
- Keep NOUNS only as they are the elements that mostly hold aspect value
- Split reviews in sentences (Avoid topic modelling in a too broad sense) (https://aclanthology.org/N10-1122.pdf)

In [ ]:
import pandas as pd
from gensim import corpora

# LDA does not benefit from repeated words among many documents. So we should clean them as they are stopwords.
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
ds = pd.read_csv(corpus_path)['comments'].apply(lambda x: x.split(' '))

dictionary = corpora.Dictionary(ds)
doc_frequency = {}

for index in range(len(dictionary)):
    doc_frequency[dictionary.get(index)] = len(ds[[dictionary.get(index) in x for x in ds]])

In [ ]:
pd.DataFrame.from_dict(doc_frequency, orient='index')[0].map(lambda x: x / len(ds)).sort_values(ascending=False)

Are 'game'(40%), 'play'(20%) and '\<game_name\>'(11%) stopwords? <br>
For sure 'game' is! What about the other two? I believe they bring no context.
We remove them!

In [1]:
stop_words = ['game', 'play', '<game_name>']

In [2]:
# Default approach on the sentences only to avoid global topic recognition
from model import LdaModelGenerator
from main.lda.config import LdaGeneratorConfig

corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
default_config = LdaGeneratorConfig(name="default")
lda, dictionary = LdaModelGenerator(default_config, stop_words).make_model(corpus_path)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/233897 [00:00<?, ?it/s]

In [3]:
from gensim.models import CoherenceModel
import pandas as pd

test_ds = pd.read_csv("../dataset/output/default_sentences/pre_processed.310k.test.csv")

# For the c_v model
texts = test_ds['comments'].apply(lambda x: x.split(' '))

topn = 10  # For the coherence evaluation.
cv_coh = CoherenceModel(lda, texts=texts, coherence='c_v', topn=topn)
npmi_coh = CoherenceModel(lda, texts=texts, coherence='c_npmi', topn=topn)

In [4]:
print(
    f"Model perplexity: {lda.log_perplexity(texts.apply(lambda x: dictionary.doc2bow(x)).tolist())}\n"
    f"With topn = {topn} we have: \n"
    f" - CV coherence: {cv_coh.get_coherence()}\n"
    f" - NPMI coherence: {npmi_coh.get_coherence()}: \n"
    "CV coherence per topic:"
)
cv_coh.get_coherence_per_topic()

Model perplexity: -31.480142717806686
With topn = 10 we have: 
 - CV coherence: 0.40848540031780045
 - NPMI coherence: 0.006683881671522012: 
CV coherence per topic:


[0.27254764741517806,
 0.4486105365601417,
 0.38699135879672514,
 0.36609561601874974,
 0.4000108767175522,
 0.4858056115886423,
 0.3388149335408939,
 0.4785679438306373,
 0.4978451360173294,
 0.5357060026370613,
 0.4060204725685014,
 0.33383785820077666,
 0.40856793362175825,
 0.3593736769352586]

In [5]:
lda.show_topics(20, 20)

[(0,
  '0.281*"player" + 0.052*"expansion" + 0.027*"try" + 0.023*"count" + 0.019*"year" + 0.017*"low" + 0.017*"well" + 0.017*"hour" + 0.017*"quickly" + 0.016*"read" + 0.016*"new" + 0.016*"old" + 0.015*"party" + 0.014*"goal" + 0.014*"good" + 0.014*"move" + 0.014*"say" + 0.013*"high" + 0.012*"figure" + 0.012*"spend"'),
 (1,
  '0.048*"turn" + 0.047*"building" + 0.045*"round" + 0.043*"end" + 0.041*"box" + 0.039*"big" + 0.038*"build" + 0.035*"work" + 0.030*"small" + 0.022*"power" + 0.019*"super" + 0.018*"choose" + 0.017*"balance" + 0.017*"couple" + 0.016*"perfect" + 0.016*"recommend" + 0.016*"create" + 0.015*"leave" + 0.015*"combo" + 0.015*"get"'),
 (2,
  '0.102*"lot" + 0.080*"bit" + 0.068*"little" + 0.055*"well" + 0.045*"take" + 0.036*"think" + 0.034*"long" + 0.028*"fun" + 0.027*"buy" + 0.026*"original" + 0.026*"get" + 0.019*"solid" + 0.017*"sure" + 0.016*"actually" + 0.014*"good" + 0.013*"similar" + 0.013*"sell" + 0.013*"thing" + 0.012*"way" + 0.011*"instead"'),
 (3,
  '0.086*"different" 

### POS: Nouns only

Now we check if the NOUN only approach works better. <br>
Take nouns only of the POS tagged ds:

In [6]:
from pre_processing import extract_pos_ds
import pandas as pd

print("Creating the __noun filtered ds:")
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.csv"
store_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

print("Creating the __noun filtered test ds:")
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.test.csv"
store_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.test.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

Creating the __noun filtered ds:


Pandas Apply:   0%|          | 0/234249 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/234249 [00:00<?, ?it/s]

ds created under: ../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv
Creating the __noun filtered test ds:


Pandas Apply:   0%|          | 0/78083 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/78083 [00:00<?, ?it/s]

ds created under: ../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.test.csv


In [ ]:
# This logic is translated in:
from pre_processing import extract_pos_ds

In [ ]:
import pandas as pd
from gensim import corpora

# LDA does not benefit from repeated words among many documents. So we should clean them as they are stopwords.
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv"
ds = pd.read_csv(corpus_path)['comments'].apply(lambda x: x.split(' '))

count_dictionary = corpora.Dictionary(ds)
doc_frequency = {}

for index in range(len(count_dictionary)):
    doc_frequency[count_dictionary.get(index)] = len(ds[[count_dictionary.get(index) in x for x in ds]])

pd.DataFrame.from_dict(doc_frequency, orient='index')[0].map(lambda x: x / len(ds))

In [4]:
# Default approach on the sentences only to avoid global topic recognition
from model import LdaModelGenerator, LdaGeneratorConfig

stop_words = ['game', 'play', '<game_name>']
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv"
default_config = LdaGeneratorConfig(name="default")
noun_lda, noun_dictionary = LdaModelGenerator(default_config, stop_words).make_model(corpus_path)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/167602 [00:00<?, ?it/s]

In [5]:
from gensim.models import CoherenceModel
import pandas as pd

test_corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.test.csv"
test_ds = pd.read_csv(test_corpus_path)

# For the c_v model
texts = test_ds['comments'].apply(lambda x: x.split(' '))

topn = 10  # For the coherence evaluation.
cv_coh = CoherenceModel(noun_lda, texts=texts, coherence='c_v', topn=topn)
npmi_coh = CoherenceModel(noun_lda, texts=texts, coherence='c_npmi', topn=topn)

In [6]:
print(
    f"Model perplexity: {noun_lda.log_perplexity(texts.apply(lambda x: noun_dictionary.doc2bow(x)).tolist())}\n"
    f"With topn = {topn} we have: \n"
    f" - CV coherence: {cv_coh.get_coherence()}\n"
    f" - NPMI coherence: {npmi_coh.get_coherence()}: \n"
    "CV coherence per topic:"
)
cv_coh.get_coherence_per_topic()

Model perplexity: -30.785678623952037
With topn = 10 we have: 
 - CV coherence: 0.2885869000600669
 - NPMI coherence: -0.03455960748217064: 
CV coherence per topic:


[0.32972073815509945,
 0.38794153062224623,
 0.2586327140975048,
 0.2619792147532881,
 0.31468470514702435,
 0.2315422886045449,
 0.19522473098956622,
 0.40443356934457364,
 0.22090880887349415,
 0.3502331299174756,
 0.28964400649838845,
 0.2626494647303481,
 0.2935958395068493,
 0.23902585960053319]

In [10]:
noun_lda.show_topics(20, 20)

[(0,
  '0.233*"bit" + 0.095*"character" + 0.095*"component" + 0.090*"box" + 0.083*"art" + 0.044*"quality" + 0.039*"depth" + 0.033*"filler" + 0.030*"twist" + 0.029*"series" + 0.027*"use" + 0.025*"count" + 0.015*"company" + 0.014*"field" + 0.012*"fantasy" + 0.012*"memory" + 0.012*"learning" + 0.012*"damage" + 0.010*"favorite" + 0.009*"curve"'),
 (1,
  '0.149*"fun" + 0.097*"lot" + 0.072*"decision" + 0.068*"placement" + 0.066*"worker" + 0.065*"people" + 0.052*"table" + 0.044*"scoring" + 0.043*"option" + 0.042*"space" + 0.035*"family" + 0.031*"opponent" + 0.025*"issue" + 0.016*"day" + 0.016*"ton" + 0.014*"light" + 0.012*"fiddly" + 0.011*"leader" + 0.011*"session" + 0.011*"review"'),
 (2,
  '0.562*"player" + 0.051*"engine" + 0.034*"goal" + 0.029*"combat" + 0.026*"ship" + 0.026*"chance" + 0.023*"movement" + 0.022*"room" + 0.020*"dungeon" + 0.017*"move" + 0.016*"difference" + 0.016*"line" + 0.015*"number" + 0.015*"skill" + 0.013*"interest" + 0.012*"situation" + 0.012*"rest" + 0.010*"position" 

In [ ]:
# Now the model has high coherence which is good but are the extracted aspects good?
# Probably not.

In [11]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.073863  0.028962       1        1  10.233599
10    -0.151808 -0.315021       2        1   8.558050
0     -0.009128  0.029131       3        1   8.301123
8     -0.036660 -0.014971       4        1   8.067015
13     0.069567  0.129207       5        1   7.652932
1     -0.257061  0.116643       6        1   7.558825
5      0.228706 -0.024492       7        1   6.851998
2      0.083933  0.163079       8        1   6.845679
4     -0.191602  0.066984       9        1   6.724631
6     -0.021336 -0.280100      10        1   6.609898
7      0.197224 -0.016674      11        1   6.374543
3      0.204400 -0.060860      12        1   6.008597
12     0.057255  0.046364      13        1   5.319983
11    -0.099625  0.131746      14        1   4.893126, topic_info=       Term         Freq        Total Category  logprob  loglift
174    card  5258.000000  5258.000000  Default  30.0000  30.0000
114  player  5176.000000  5176.000000  Default  29.0000  29.0000
2      like  2674.000000  2674.000000  Default  28.0000  28.0000
133    time  2661.000000  2661.000000  Default  27.0000  27.0000
56     feel  1545.000000  1545.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
269    know   147.456546   416.027141  Topic14  -4.3008   1.9801
147    good   187.249148  3180.132809  Topic14  -4.0619   0.1851
165     way   139.221540  1260.276475  Topic14  -4.3583   0.8143
265   think   128.326255  1317.980633  Topic14  -4.4397   0.6881
158    need    91.832623  1164.392087  Topic14  -4.7744   0.4774

[557 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
694      10  1.000856    <number>
616       2  1.001130     ability
217      14  1.000692        able
1079     13  1.001963  absolutely
865       7  0.998053    abstract
...     ...       ...         ...
686      12  0.998459       worth
1056      9  0.998925       write
1496      7  0.999617           x
207       3  0.998728        year
1430     11  0.996514       young

[594 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 11, 1, 9, 14, 2, 6, 3, 5, 7, 8, 4, 13, 12])

In [12]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(noun_lda, texts.apply(lambda x: noun_dictionary.doc2bow(x)).tolist(), noun_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
1     -0.220023  0.046793       1        1  9.237302
4     -0.124122 -0.171847       2        1  8.824796
5      0.147753  0.239375       3        1  7.917529
7      0.097326 -0.290005       4        1  7.780364
2      0.180887  0.220143       5        1  7.655149
12     0.140260 -0.046130       6        1  7.462457
6      0.127570 -0.115383       7        1  7.276919
9     -0.144675  0.043897       8        1  7.045944
13    -0.101482  0.036159       9        1  6.960874
10    -0.128088  0.041222      10        1  6.921349
0      0.138861 -0.044043      11        1  6.311439
11    -0.124049  0.040124      12        1  6.040525
8     -0.125976  0.039638      13        1  5.412898
3      0.135756 -0.039943      14        1  5.152456, topic_info=              Term         Freq        Total Category  logprob  loglift
11          player  9480.000000  9480.000000  Default  30.0000  30.0000
78            card  9089.000000  9089.000000  Default  29.0000  29.0000
56            time  5034.000000  5034.000000  Default  28.0000  28.0000
45             bit  3243.000000  3243.000000  Default  27.0000  27.0000
25            rule  3202.000000  3202.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
749           help    81.095791    81.151846  Topic14  -4.9421   2.9650
569       painting    78.136667    78.192723  Topic14  -4.9792   2.9650
198           hang    75.785980    75.842036  Topic14  -5.0098   2.9650
554  consideration    74.759676    74.815732  Topic14  -5.0234   2.9649
748      equipment    68.241240    68.297296  Topic14  -5.1147   2.9649

[467 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
252       2  0.999756   ability
593       6  0.998159  abstract
337       6  0.999875    action
331       9  0.995365  activity
469       6  1.001131  addition
...     ...       ...       ...
2         3  1.000679      work
350       1  1.000308    worker
131       4  1.000933     world
639      13  0.997347         x
181       4  1.000086      year

[437 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 6, 8, 3, 13, 7, 10, 14, 11, 1, 12, 9, 4])

In [ ]:
# Topic 1 identifies Downtime for example. Yet it also does crash with maybe with bookkeeping.
# We have to better tune the parameters to check if we can create better communities

Following the experiments I decided to follow the path of nouns only composition document. <br>
The hyperparameters tuning is now the next step.

Noun only on sentences model performs way worse, let's see if going for non LocalLDA but focusing on NOUNS only is viable:

In [13]:
from pre_processing import extract_pos_ds
import pandas as pd

print("Creating the __noun filtered ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

print("Creating the __noun filtered test ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.test.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

Creating the __noun filtered ds:


Pandas Apply:   0%|          | 0/233777 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/233777 [00:00<?, ?it/s]

ds created under: ../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv
Creating the __noun filtered test ds:


Pandas Apply:   0%|          | 0/77926 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/77926 [00:00<?, ?it/s]

ds created under: ../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv


In [14]:
# Default approach on the sentences only to avoid global topic recognition
from model import LdaModelGenerator, LdaGeneratorConfig

stop_words = ['game', 'play', '<game_name>']
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
default_config = LdaGeneratorConfig(name='default')
noun_lda, noun_dictionary = LdaModelGenerator(default_config, stop_words).make_model(corpus_path)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/195789 [00:00<?, ?it/s]

In [15]:
from gensim.models import CoherenceModel
import pandas as pd

test_corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"
test_ds = pd.read_csv(test_corpus_path)

# For the c_v model
texts = test_ds['comments'].apply(lambda x: x.split(' '))

topn = 10  # For the coherence evaluation.
cv_coh = CoherenceModel(noun_lda, texts=texts, coherence='c_v', topn=topn)
npmi_coh = CoherenceModel(noun_lda, texts=texts, coherence='c_npmi', topn=topn)

In [16]:
print(
    f"Model perplexity: {noun_lda.log_perplexity(texts.apply(lambda x: noun_dictionary.doc2bow(x)).tolist())}\n"
    f"With topn = {topn} we have: \n"
    f" - CV coherence: {cv_coh.get_coherence()}\n"
    f" - NPMI coherence: {npmi_coh.get_coherence()}: \n"
    "CV coherence per topic:"
)
cv_coh.get_coherence_per_topic()

Model perplexity: -23.98261823214068
With topn = 10 we have: 
 - CV coherence: 0.5975213978908255
 - NPMI coherence: 0.009577051016547825: 
CV coherence per topic:


[0.5551108165702432,
 0.713715061668484,
 0.5962325548829728,
 0.7550304784445258,
 0.6024835953397344,
 0.532905401010275,
 0.5991436978554352,
 0.47656312764646813,
 0.6019909845184954,
 0.5737234016458104,
 0.6269564511945285,
 0.58992218149521,
 0.6158909102359408,
 0.5256309079634304]

In [17]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(noun_lda, texts.apply(lambda x: noun_dictionary.doc2bow(x)).tolist(), noun_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.183221 -0.024029       1        1  10.567187
1     -0.202968 -0.116593       2        1  10.484280
4     -0.180284  0.147250       3        1   9.277482
6     -0.192147 -0.251542       4        1   9.107217
2      0.165870  0.028396       5        1   7.696528
11     0.164678  0.099495       6        1   7.498959
5     -0.045723  0.258772       7        1   7.154036
7      0.078403  0.059528       8        1   6.212306
13    -0.131547  0.212850       9        1   6.201236
12     0.016073 -0.222965      10        1   6.195035
10     0.248552 -0.094541      11        1   5.617402
0      0.089321 -0.106211      12        1   5.101056
9      0.087569 -0.015906      13        1   4.642338
8      0.085424  0.025495      14        1   4.244939, topic_info=       Term          Freq         Total Category  logprob  loglift
87     card  20422.000000  20422.000000  Default  30.0000  30.0000
9    player  23063.000000  23063.000000  Default  29.0000  29.0000
28     rule   8386.000000   8386.000000  Default  28.0000  28.0000
97      fun   9773.000000   9773.000000  Default  27.0000  27.0000
38    theme   7768.000000   7768.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
212   skill    452.898385    740.202294  Topic14  -3.9801   2.6682
221   depth    660.682224   1458.743739  Topic14  -3.6025   2.3674
472  course    204.600841    493.505163  Topic14  -4.7747   2.2790
77     luck    389.086867   4206.965654  Topic14  -4.1320   0.7787
14      bit    248.353330   7763.470866  Topic14  -4.5809  -0.2829

[631 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
293       1  0.483803   ability
293      11  0.516137   ability
650       7  1.000373  abstract
382       2  1.000045    action
371       6  1.000030  activity
...     ...       ...       ...
155       8  0.188026     world
155      11  0.560722     world
646      11  0.994127     wrong
691      11  1.000770         x
29       13  0.999793      year

[697 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 7, 3, 12, 6, 8, 14, 13, 11, 1, 10, 9])